In [1]:
import pandas as pd

df = pd.read_csv('Kuperman-BRM-data-2012.csv')

In [2]:
import numpy as np

# Read the content of the file
with open('sotu/Aggregated_Speeches/Bush_sr.txt', 'r') as file:
    text = file.read()

# Tokenize the text into words
words = text.split()

# Initialize a list to store the Rating.Mean values
ratings = []

# Cross-check each word with the dataframe and record the Rating.Mean
for word in words:
    rating = df.loc[df['Word'] == word, 'Rating.Mean']
    if not rating.empty:
        ratings.append(rating.values[0])

# Calculate the average Rating.Mean
average_rating_mean = np.mean(ratings) if ratings else float('nan')

# Print the average Rating.Mean
print(f'Average Rating.Mean: {average_rating_mean}')

Average Rating.Mean: 6.980026749260875


In [4]:
# import numpy as np
# import pandas as pd
# import nltk
# from nltk.tokenize import sent_tokenize
# import string

# # Ensure you have the necessary NLTK data files
# # nltk.download('punkt')

# # Example dataframe setup (remove this if you already have df defined)
# # data = {'Word': ['freedom', 'peace', 'America', 'security'], 'Rating.Mean': [4.5, 3.8, 4.2, 4.0]}
# # df = pd.DataFrame(data)

# # Read the content of the file
# with open('sotu/Aggregated_Speeches/Bush_sr.txt', 'r', encoding='utf-8') as file:
#     text = file.read()

# # Tokenize the text into sentences
# sentences = sent_tokenize(text)

# # Initialize lists to store sentence ratings and corresponding sentences
# sentence_ratings = []
# rated_sentences = []

# # Function to clean and normalize words
# def clean_word(word):
#     return word.strip(string.punctuation).lower()

# # Iterate over each sentence
# for sentence in sentences:
#     # Tokenize the sentence into words
#     words = sentence.split()
    
#     # Initialize a list to store Rating.Mean values for the current sentence
#     ratings = []
    
#     # Iterate over each word in the sentence
#     for word in words:
#         cleaned_word = clean_word(word)
#         # Retrieve the Rating.Mean for the word from the dataframe
#         rating = df.loc[df['Word'].str.lower() == cleaned_word, 'Rating.Mean']
#         if not rating.empty:
#             ratings.append(rating.values[0])
    
#     # Calculate the average Rating.Mean for the sentence
#     average_rating = np.mean(ratings) if ratings else float('nan')
    
#     # Append the results to the lists
#     sentence_ratings.append(average_rating)
#     rated_sentences.append(sentence)

# # Create a dataframe to hold sentences and their average ratings
# results_df = pd.DataFrame({
#     'Sentence': rated_sentences,
#     'Average_Rating_Mean': sentence_ratings
# })

# # Calculate the overall average Rating.Mean for the entire script
# overall_average = results_df['Average_Rating_Mean'].mean()

# # Find the sentence with the highest Rating.Mean
# highest_rated = results_df.loc[results_df['Average_Rating_Mean'].idxmax()]

# # Find the sentence with the lowest Rating.Mean
# lowest_rated = results_df.loc[results_df['Average_Rating_Mean'].idxmin()]

# # Print the results
# print(f'Overall Average Rating.Mean: {overall_average:.2f}\n')

# print('Sentence with the Highest Rating.Mean:')
# print(f'Average Rating.Mean: {highest_rated["Average_Rating_Mean"]:.2f}')
# print(f'Sentence: {highest_rated["Sentence"]}\n')

# print('Sentence with the Lowest Rating.Mean:')
# print(f'Average Rating.Mean: {lowest_rated["Average_Rating_Mean"]:.2f}')
# print(f'Sentence: {lowest_rated["Sentence"]}')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joshstrupp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Overall Average Rating.Mean: 7.06

Sentence with the Highest Rating.Mean:
Average Rating.Mean: 15.06
Sentence: And instead of 16 workers paying in for every beneficiary,
right now it's only about 3 workers.

Sentence with the Lowest Rating.Mean:
Average Rating.Mean: 2.89
Sentence: And so tonight, I'm going to ask something of every one of you.


In [6]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import string

# Ensure you have the necessary NLTK data files
# nltk.download('punkt')

# Load the AoA data
df_aoa = pd.read_csv('Kuperman-BRM-data-2012.csv')

# Create a mapping from words to their Rating.Mean
word_to_rating = dict(zip(df_aoa['Word'].str.lower(), df_aoa['Rating.Mean']))

# Define the path to the speeches
speech_folder = 'sotu/Aggregated_Speeches/'

# Get a list of all speech files
speech_files = [f for f in os.listdir(speech_folder) if f.endswith('.txt')]

# Define a translator to remove punctuation
translator = str.maketrans('', '', string.punctuation)

# Initialize a list to store results
results = []

for speech_file in speech_files:
    # Extract the president's name from the file name
    president_name = speech_file.replace('.txt', '')

    # Read the speech text
    with open(os.path.join(speech_folder, speech_file), 'r', encoding='utf-8') as f:
        text = f.read()

    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove punctuation and make all words lowercase
    words = [word.translate(translator).lower() for word in words if word.translate(translator).isalpha()]

    # Get the Rating.Mean for each word
    word_ratings = [(word, word_to_rating[word]) for word in words if word in word_to_rating]

    if not word_ratings:
        # Skip if no words have a Rating.Mean
        continue

    # Calculate the average AoA
    ratings = [rating for word, rating in word_ratings]
    avg_AoA = np.mean(ratings)

    # Find the 5 unique words with the highest Rating.Mean
    biggest_words = []
    smallest_words = []
    seen_words = set()

    for word, rating in sorted(word_ratings, key=lambda x: x[1], reverse=True):
        if word not in seen_words:
            biggest_words.append(word)
            seen_words.add(word)
        if len(biggest_words) == 5:
            break

    seen_words.clear()

    # Find the 5 unique words with the lowest Rating.Mean
    for word, rating in sorted(word_ratings, key=lambda x: x[1]):
        if word not in seen_words:
            smallest_words.append(word)
            seen_words.add(word)
        if len(smallest_words) == 5:
            break

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Initialize lists to store sentences and their AoA sums
    sentence_sums = []

    for sentence in sentences:
        # Tokenize the sentence into words
        sentence_words = word_tokenize(sentence)
        # Remove punctuation and make all words lowercase
        sentence_words = [word.translate(translator).lower() for word in sentence_words if word.translate(translator)]

        # Check if the sentence meets the length criteria
        if 10 <= len(sentence_words) < 20:
            # Get the Rating.Mean for each word in the sentence
            ratings_in_sentence = [word_to_rating[word] for word in sentence_words if word in word_to_rating]
            if ratings_in_sentence:
                sum_ratings = sum(ratings_in_sentence)
                sentence_sums.append((sentence, sum_ratings))

    if sentence_sums:
        # Find the sentence with the highest sum of AoA ratings
        biggest_sentence = max(sentence_sums, key=lambda x: x[1])[0]
        # Find the sentence with the lowest sum of AoA ratings
        smallest_sentence = min(sentence_sums, key=lambda x: x[1])[0]
    else:
        biggest_sentence = None
        smallest_sentence = None

    # Append the results to the list
    results.append({
        'President': president_name,
        'avg_AoA': avg_AoA,
        'biggest_words': biggest_words,
        'smallest_words': smallest_words,
        'biggest_sentence': biggest_sentence,
        'smallest_sentence': smallest_sentence
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joshstrupp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


          President   avg_AoA  \
0             Nixon  7.122861   
1             Hayes  8.135259   
2            Monroe  7.929597   
3          Coolidge  7.960240   
4    Johnson_Andrew  7.654973   
5            Taylor  8.161355   
6             Buren  8.053105   
7           Bush_sr  7.143724   
8           Clinton  6.844751   
9           Jackson  7.987420   
10  Roosevelt_Teddy  7.623287   
11           Reagan  7.277296   
12           Hoover  8.156791   
13          Bush_jr  7.219216   
14           Carter  7.986420   
15     Roosevelt_FD  7.410962   
16          Kennedy  7.615822   
17        Jefferson  7.796499   
18         Fillmore  7.978290   
19         Harrison  7.903685   
20             Ford  7.573697   
21       Washington  8.098030   
22          Harding  8.100143   
23            Grant  7.943165   
24          Adams_J  8.132742   
25          Lincoln  7.803928   
26         McKinley  8.178186   
27            Trump  6.989892   
28         Buchanan  7.944884   
29       E

In [7]:
import os
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import string
from nltk.corpus import wordnet as wn

# Ensure you have the necessary NLTK data files
# nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load the AoA data
df_aoa = pd.read_csv('Kuperman-BRM-data-2012.csv')

# Create a mapping from words to their Rating.Mean
word_to_rating = dict(zip(df_aoa['Word'].str.lower(), df_aoa['Rating.Mean']))

# Define the path to the speeches
speech_folder = 'sotu/Aggregated_Speeches/'

# Get a list of all speech files
speech_files = [f for f in os.listdir(speech_folder) if f.endswith('.txt')]

# Define a translator to remove punctuation
translator = str.maketrans('', '', string.punctuation)

# Function to get definitions using WordNet
def get_definition(word):
    synsets = wn.synsets(word)
    if synsets:
        return synsets[0].definition()
    else:
        return None

# Function to compute average AoA of a sentence
def compute_average_aoa(sentence):
    # Tokenize the sentence into words
    sentence_words = word_tokenize(sentence)
    # Remove punctuation and make all words lowercase
    sentence_words = [word.translate(translator).lower() for word in sentence_words if word.translate(translator).isalpha()]
    # Get the Rating.Mean for each word in the sentence
    ratings_in_sentence = [word_to_rating[word] for word in sentence_words if word in word_to_rating]
    if ratings_in_sentence:
        avg_rating = np.mean(ratings_in_sentence)
    else:
        avg_rating = None
    return avg_rating

# Initialize a list to store results
results = []

for speech_file in speech_files:
    # Extract the president's name from the file name
    president_name = speech_file.replace('.txt', '')

    # Read the speech text
    with open(os.path.join(speech_folder, speech_file), 'r', encoding='utf-8') as f:
        text = f.read()

    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove punctuation and make all words lowercase
    words = [word.translate(translator).lower() for word in words if word.translate(translator).isalpha()]

    # Get the Rating.Mean for each word
    word_ratings = [(word, word_to_rating[word]) for word in words if word in word_to_rating]

    if not word_ratings:
        # Skip if no words have a Rating.Mean
        continue

    # Calculate the average AoA
    ratings = [rating for word, rating in word_ratings]
    avg_AoA = np.mean(ratings)

    # Find the 5 unique words with the highest Rating.Mean
    biggest_words = []
    biggest_words_definitions = []
    seen_words = set()

    for word, rating in sorted(word_ratings, key=lambda x: x[1], reverse=True):
        if word not in seen_words:
            biggest_words.append(word)
            definition = get_definition(word)
            biggest_words_definitions.append(definition)
            seen_words.add(word)
        if len(biggest_words) == 5:
            break

    seen_words.clear()

    # Find the 5 unique words with the lowest Rating.Mean
    smallest_words = []
    smallest_words_definitions = []
    for word, rating in sorted(word_ratings, key=lambda x: x[1]):
        if word not in seen_words:
            smallest_words.append(word)
            definition = get_definition(word)
            smallest_words_definitions.append(definition)
            seen_words.add(word)
        if len(smallest_words) == 5:
            break

    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Initialize lists to store sentences and their AoA sums
    sentence_sums = []

    for idx, sentence in enumerate(sentences):
        # Tokenize the sentence into words
        sentence_words = word_tokenize(sentence)
        # Remove punctuation and make all words lowercase
        sentence_words = [word.translate(translator).lower() for word in sentence_words if word.translate(translator)]

        # Check if the sentence meets the length criteria
        if 10 <= len(sentence_words) < 20:
            # Get the Rating.Mean for each word in the sentence
            ratings_in_sentence = [word_to_rating[word] for word in sentence_words if word in word_to_rating]
            if ratings_in_sentence:
                sum_ratings = sum(ratings_in_sentence)
                sentence_sums.append((idx, sentence, sum_ratings))

    if sentence_sums:
        # Find the sentence with the highest sum of AoA ratings
        max_entry = max(sentence_sums, key=lambda x: x[2])
        biggest_sentence_idx, biggest_sentence, _ = max_entry

        # Find the sentence with the lowest sum of AoA ratings
        min_entry = min(sentence_sums, key=lambda x: x[2])
        smallest_sentence_idx, smallest_sentence, _ = min_entry

        # Compute average AoA for biggest_sentence and smallest_sentence
        biggest_sentence_rating = compute_average_aoa(biggest_sentence)
        smallest_sentence_rating = compute_average_aoa(smallest_sentence)

        # Get sentences before and after biggest_sentence
        if biggest_sentence_idx > 0:
            biggest_sentence_before = sentences[biggest_sentence_idx - 1]
        else:
            biggest_sentence_before = None
        if biggest_sentence_idx < len(sentences) - 1:
            biggest_sentence_after = sentences[biggest_sentence_idx + 1]
        else:
            biggest_sentence_after = None

        # Get sentences before and after smallest_sentence
        if smallest_sentence_idx > 0:
            smallest_sentence_before = sentences[smallest_sentence_idx - 1]
        else:
            smallest_sentence_before = None
        if smallest_sentence_idx < len(sentences) - 1:
            smallest_sentence_after = sentences[smallest_sentence_idx + 1]
        else:
            smallest_sentence_after = None
    else:
        biggest_sentence = None
        smallest_sentence = None
        biggest_sentence_rating = None
        smallest_sentence_rating = None
        biggest_sentence_before = None
        biggest_sentence_after = None
        smallest_sentence_before = None
        smallest_sentence_after = None

    # Append the results to the list
    results.append({
        'President': president_name,
        'avg_AoA': avg_AoA,
        'biggest_words': biggest_words,
        'biggest_words_definitions': biggest_words_definitions,
        'smallest_words': smallest_words,
        'smallest_words_definitions': smallest_words_definitions,
        'biggest_sentence': biggest_sentence,
        'biggest_sentence_rating': biggest_sentence_rating,
        'biggest_sentence_before': biggest_sentence_before,
        'biggest_sentence_after': biggest_sentence_after,
        'smallest_sentence': smallest_sentence,
        'smallest_sentence_rating': smallest_sentence_rating,
        'smallest_sentence_before': smallest_sentence_before,
        'smallest_sentence_after': smallest_sentence_after
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joshstrupp/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/joshstrupp/nltk_data...


          President   avg_AoA  \
0             Nixon  7.122861   
1             Hayes  8.135259   
2            Monroe  7.929597   
3          Coolidge  7.960240   
4    Johnson_Andrew  7.654973   
5            Taylor  8.161355   
6             Buren  8.053105   
7           Bush_sr  7.143724   
8           Clinton  6.844751   
9           Jackson  7.987420   
10  Roosevelt_Teddy  7.623287   
11           Reagan  7.277296   
12           Hoover  8.156791   
13          Bush_jr  7.219216   
14           Carter  7.986420   
15     Roosevelt_FD  7.410962   
16          Kennedy  7.615822   
17        Jefferson  7.796499   
18         Fillmore  7.978290   
19         Harrison  7.903685   
20             Ford  7.573697   
21       Washington  8.098030   
22          Harding  8.100143   
23            Grant  7.943165   
24          Adams_J  8.132742   
25          Lincoln  7.803928   
26         McKinley  8.178186   
27            Trump  6.989892   
28         Buchanan  7.944884   
29       E

In [8]:
results_df.to_csv('results.csv', index=False)

In [9]:
import pandas as pd
import ast

# Read the existing results CSV
results_df = pd.read_csv('results2.csv')

# Read the AoA data
df_aoa = pd.read_csv('Kuperman-BRM-data-2012.csv')

# Create a mapping from words to their Rating.Mean
word_to_rating = dict(zip(df_aoa['Word'].str.lower(), df_aoa['Rating.Mean']))

# Function to parse the list of words from the DataFrame column (strings)
def parse_word_list(word_list_str):
    try:
        # Use ast.literal_eval to safely parse the string representation of the list
        word_list = ast.literal_eval(word_list_str)
        return word_list
    except:
        return []

# Initialize lists to store the average ratings
avg_biggest_words_rating_list = []
avg_smallest_words_rating_list = []

for idx, row in results_df.iterrows():
    # Parse the biggest_words and smallest_words
    biggest_words = parse_word_list(row['biggest_words'])
    smallest_words = parse_word_list(row['smallest_words'])

    # Calculate average Rating.Mean for biggest_words
    ratings = []
    for word in biggest_words:
        word_lower = word.lower()
        if word_lower in word_to_rating:
            ratings.append(word_to_rating[word_lower])
    if ratings:
        avg_biggest_words_rating = sum(ratings) / len(ratings)
    else:
        avg_biggest_words_rating = None
    avg_biggest_words_rating_list.append(avg_biggest_words_rating)

    # Calculate average Rating.Mean for smallest_words
    ratings = []
    for word in smallest_words:
        word_lower = word.lower()
        if word_lower in word_to_rating:
            ratings.append(word_to_rating[word_lower])
    if ratings:
        avg_smallest_words_rating = sum(ratings) / len(ratings)
    else:
        avg_smallest_words_rating = None
    avg_smallest_words_rating_list.append(avg_smallest_words_rating)

# Add new columns to DataFrame
results_df['avg_biggest_words_rating'] = avg_biggest_words_rating_list
results_df['avg_smallest_words_rating'] = avg_smallest_words_rating_list

# Save updated DataFrame to CSV
results_df.to_csv('results2_updated.csv', index=False)

# Display updated DataFrame
print(results_df)

                 President   avg_AoA  \
0            Richard Nixon  7.122861   
1      Rutherford B. Hayes  8.135259   
2             James Monroe  7.929597   
3          Calvin Coolidge  7.960240   
4           Andrew Johnson  7.654973   
5           Zachary Taylor  8.161355   
6         Martin Van Buren  8.053105   
7          George Bush Sr.  7.143724   
8             Bill Clinton  6.844751   
9           Andrew Jackson  7.987420   
10         Teddy Roosevelt  7.623287   
11           Ronald Reagan  7.277296   
12          Herbert Hoover  8.156791   
13          George W. Bush  7.219216   
14            Jimmy Carter  7.986420   
15   Franklin D. Roosevelt  7.410962   
16         John F. Kennedy  7.615822   
17        Thomas Jefferson  7.796499   
18        Millard Fillmore  7.978290   
19  William Henry Harrison  7.903685   
20             Gerald Ford  7.573697   
21       George Washington  8.098030   
22       Warren G. Harding  8.100143   
23        Ulysses S. Grant  7.943165   
